In [ ]:
# %load ../helpers.py
%matplotlib inline
%env NI_PAGER="cat"
import pandas as pd
import numpy as np
from os.path import join, split, exists, splitext


In [ ]:
from collections import defaultdict
from bokeh.io import show, save
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper
)
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure
from shapely.geometry import box, shape
from shapely.ops import cascaded_union, unary_union

INPUT_FOLDER = 'geohash_data'

In [ ]:
def make_box(n, s, e, w):
    xs = [n, n, s, s]
    ys = [e, w, w, e]
    return xs, ys

def preprocess_geohash_data(input_fn, output_fn):
    !ni {input_fn} gAu p'r a, ghb(a), murmurhash3(b), FR 1' \>{output_fn}
    return

In [ ]:
raw_gh_info_fn = join(INPUT_FOLDER, 'dma_gh4s')
color_fn = join(INPUT_FOLDER, 'dma_gh4s_color')
preprocess_geohash_data(raw_gh_info_fn, color_fn)
input_df = pd.read_csv(color_fn, header=None, sep='\t')
print input_df.head()
input_df.columns=['gh', 'N', 'S', 'E', 'W', 'color', 'dma_name', 'dma_id']
input_df = input_df.groupby(['gh']).first().reset_index()
input_df.head()

data_dict = defaultdict(list)

for idx, row in input_df.iterrows():
    gh= row['gh']
    raw_dma = row['dma_name']
    color = row['color']
    red = color & 255
    green = (color >> 8) & 255
    blue = (color >> 16) & 255
#     print color, red, green, blue
    
    ys, xs = make_box(row['N'], row['S'], row['E'], row['W'])
    
    
    dma_pieces = raw_dma.split('__')[-1].split('_')
    dma = '{}, {}'.format(' '.join(dma_pieces[:-1]), dma_pieces[-1])
    
    data_dict['x'].append(xs)
    data_dict['y'].append(ys)
    data_dict['dma'].append(dma)
    data_dict['gh'].append(gh)
    data_dict['color'].append("#{:02X}{:02X}{:02X}".format(red, green, blue))

source = ColumnDataSource(data=data_dict)

TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

p = figure(
    title="US DMA Geohash-4s", tools=TOOLS,
    x_axis_location=None, y_axis_location=None
)
p.grid.grid_line_color = None

p.patches('x', 'y', source=source,
          fill_color={'field': 'color'},
          fill_alpha=0.7, line_color="white", line_width=0.1)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("DMA Name", "@dma"),
    ("Geohash", "@gh"),
    ("Latitude", "$y"),
    ("Longitude", "$x")
]

show(p)
save(p, join(INPUT_FOLDER, 'dma_colors.html'))